## NVIDIA

In [ ]:
import re

filepath = "/workspace/DeepBench/code/nvidia/db_nvidia_resnext.txt"

data = []
with open(filepath, 'r') as file:
    # this file is usually small => can read all at once
    lines = file.readlines()
#     headers = lines[3]
    lines = lines[5:]
    # w,h,c,n,k,f_w,f_h,pad_w,pad_h,stride_w,stride_h,group,precision,fwd_time(usec),bw_inputs_time(usec),bwd_params_time(usec),total_time(usec),fwd_algo,bwd_inputs_algo,bwd_params_algo
    pattern = re.compile(r'\s*(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\w+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\w+)\s+(\w+)\s+(\w+)\s*')
    for line in lines:
        info = pattern.match(line)
#         print(info.groups())
        data.append(info.groups())

In [ ]:
import pandas as pd

times = pd.DataFrame(data, columns=["w","h","c","n","k","f_w","f_h","pad_w","pad_h","stride_w","stride_h","group","precision", \
                                    "fwd_time (usec)","bwd_inputs_time (usec)","bwd_params_time (usec)","total_time (usec)", \
                                    "fwd_algo","bwd_inputs_algo","bwd_params_algo"])

numeric_cols = [col for col in times.columns.tolist() if col not in ['precision', 'fwd_algo', 'bwd_inputs_algo', 'bwd_params_algo']]
times[numeric_cols] = times[numeric_cols].apply(pd.to_numeric)
print(times)

In [ ]:
# times_resnext101_32x48d = times.loc[times['n']==2]
times_resnext101_32x8d = times.loc[times['n']==28]
display(times_resnext101_32x8d)

In [ ]:
# archinfo = pd.read_pickle('/workspace/resnext101_32x48d-archinfo.pkl')
archinfo = pd.read_pickle('/workspace/resnext101_32x8d-archinfo.pkl')

# archinfo['n'].replace(4, 2,inplace=True)
print(archinfo)

In [ ]:
# took me > 1 hour to write this line of code. fml
diffconvs = archinfo.groupby(archinfo.columns.tolist(),as_index=False).size().reset_index().rename(columns={0:'repetitions'})
if 'dil_h' in archinfo.columns:
    diffconvs.drop(['dil_h'], axis=1, inplace=True)
if 'dil_w' in archinfo.columns:
    diffconvs.drop(['dil_w'], axis=1, inplace=True)
if 'fwd_algo' in archinfo.columns:
    diffconvs.drop(['fwd_algo'], axis=1, inplace=True)
print(diffconvs)

In [ ]:
joincols = [col for col in diffconvs.columns.tolist() if col != "repetitions"]
model_total = pd.merge(times_resnext101_32x8d, diffconvs, on=joincols, how='inner')
model_total["fwd_time_with_reps(usec)"] = model_total["fwd_time (usec)"] * model_total["repetitions"]
model_total["bwd_inputs_time_with_reps(usec)"] = model_total["bwd_inputs_time (usec)"] * model_total["repetitions"]
model_total["bwd_params_time_with_reps(usec)"] = model_total["bwd_params_time (usec)"] * model_total["repetitions"]
model_total["total_time_with_reps(usec)"] = model_total["total_time (usec)"] * model_total["repetitions"]
display(model_total)

In [ ]:
large_time=model_total[model_total['Total Time with repetitions (msec)']>100]
display(large_time)

In [ ]:
model_total.to_csv('/workspace/DeepLearningMisc/db_nv_resnext101-32x8d_clk1072_total.csv')

## AMD

In [ ]:
import re

filepath = "/DeepBench/code/amd/db_amd_resnext.txt"

data = []
with open(filepath, 'r') as file:
    # this file is usually small => can read all at once
    lines = file.readlines()
#     headers = lines[3]
    lines = lines[4:-1]
    # w,h,c,n,k,f_w,f_h,pad_w,pad_h,stride_w,stride_h,group,fwd_time(usec),bw_inputs_time(usec),bwd_params_time(usec),total_time(usec),fwd_algo,bwd_inputs_algo,bwd_params_algo
    pattern = re.compile(r'\s*(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\w+)\s+(\w+)\s+(\w+)\s*')
    for line in lines:
        info = pattern.match(line)
#         print(info.groups())
        data.append(info.groups())
print(len(data))

In [ ]:
import pandas as pd

times = pd.DataFrame(data, columns=["w","h","c","n","k","f_w","f_h","pad_w","pad_h","stride_w","stride_h","group", \
                                    "fwd_time (usec)","bwd_inputs_time (usec)","bwd_params_time (usec)","total_time (usec)", \
                                    "fwd_algo","bwd_inputs_algo","bwd_params_algo"])

numeric_cols = [col for col in times.columns.tolist() if col not in ['fwd_algo', 'bwd_inputs_algo', 'bwd_params_algo']]
times[numeric_cols] = times[numeric_cols].apply(pd.to_numeric)
print(times)

In [ ]:
# times_resnext101_32x48d = times.loc[times['n']==2]
times_resnext101_32x8d = times.loc[times['n']==28]
display(times_resnext101_32x8d)

In [ ]:
# archinfo = pd.read_pickle('resnext101_32x48d-archinfo.pkl')
archinfo = pd.read_pickle('resnext101_32x8d-archinfo.pkl')

# archinfo['n'].replace(4, 2,inplace=True)
print(archinfo)

In [ ]:
# took me > 1 hour to write this line of code. fml
diffconvs = archinfo.groupby(archinfo.columns.tolist(),as_index=False).size().reset_index().rename(columns={0:'repetitions'})
if 'dil_h' in archinfo.columns:
    diffconvs.drop(['dil_h'], axis=1, inplace=True)
if 'dil_w' in archinfo.columns:
    diffconvs.drop(['dil_w'], axis=1, inplace=True)
if 'fwd_algo' in archinfo.columns:
    diffconvs.drop(['fwd_algo'], axis=1, inplace=True)
print(diffconvs)

In [ ]:
joincols = [col for col in diffconvs.columns.tolist() if col != "repetitions"]
model_total = pd.merge(times_resnext101_32x8d, diffconvs, on=joincols, how='inner')
model_total["fwd_time_with_reps(usec)"] = model_total["fwd_time (usec)"] * model_total["repetitions"]
model_total["bwd_inputs_time_with_reps(usec)"] = model_total["bwd_inputs_time (usec)"] * model_total["repetitions"]
model_total["bwd_params_time_with_reps(usec)"] = model_total["bwd_params_time (usec)"] * model_total["repetitions"]
model_total["total_time_with_reps(usec)"] = model_total["total_time (usec)"] * model_total["repetitions"]
display(model_total)

In [ ]:
model_total.to_csv('/DeepLearningMisc/db_amd_resnext101-32x8d_mclk877_total.csv')